In [3]:
import os
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt

from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform
from sklearn.covariance import GraphicalLasso

import torch
from torch_geometric.data import InMemoryDataset
from torch_geometric.data import Data

import hypernetx as hnx
import hypernetx.algorithms.hypergraph_modularity as hmod
import networkx as nx
from networkx.convert_matrix import from_numpy_array

import functions as f
from functions import dict_to_array, normalize_array

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/mathilde/Documents/Bachelor-Thesis-Alzheimers-HGNN/functions.py:454: DeprecationWarning: invalid escape sequence '\m'
  """The hypergraph convolutional operator from the `"Hypergraph Convolution


In [4]:
# Loading atlas, time series and threshold
_, _, atlas_labels, n_ROIs = f.gen_atlas_labels()

In [5]:
# Loading the time series
time_series_list = f.load_time_series()
time_series_df_list = f.list_of_df_of_time_series(time_series_list)
# time_series_list[0]
# time_series_df_list[0]

In [6]:
# Loading the pearson correlation matrices
corr_matrix_list = f.corr_matrix_paths()['pearson']
print(corr_matrix_list)

['patient_136_S_4517.csv', 'patient_130_S_4468.csv', 'patient_002_S_4654.csv', 'patient_002_S_4251.csv', 'patient_013_S_5171.csv', 'patient_130_S_2391.csv', 'patient_031_S_4947.csv', 'patient_100_S_5102.csv', 'patient_012_S_5121.csv', 'patient_053_S_5287.csv', 'patient_002_S_1261.csv', 'patient_018_S_4313.csv', 'patient_130_S_4641.csv', 'patient_012_S_4188.csv', 'patient_002_S_4521.csv', 'patient_130_S_4250.csv', 'patient_031_S_4005.csv', 'patient_130_S_5006.csv', 'patient_006_S_4363.csv', 'patient_012_S_4012.csv', 'patient_031_S_4149.csv', 'patient_031_S_4029.csv', 'patient_013_S_4917.csv', 'patient_129_S_4287.csv', 'patient_130_S_4730.csv', 'patient_129_S_0778.csv', '.DS_Store', 'patient_031_S_4203.csv', 'patient_013_S_4268.csv', 'patient_053_S_0919.csv', 'patient_002_S_4447.csv', 'patient_012_S_4987.csv', 'patient_018_S_4868.csv', 'patient_100_S_4556.csv', 'patient_100_S_5106.csv', 'patient_130_S_5175.csv', 'patient_130_S_4294.csv', 'patient_053_S_5296.csv', 'patient_031_S_4474.csv'

In [7]:
path = f'ADNI_full/corr_matrices/corr_matrix_pearson/{corr_matrix_list[0]}'
corr_matrix_patient = np.loadtxt(path, delimiter=',')
print(corr_matrix_patient)

[[ 1.          0.62126391  0.40501965 ... -0.07884583  0.07740484
  -0.36456412]
 [ 0.62126391  1.         -0.0033939  ...  0.02684759  0.0412815
  -0.1746839 ]
 [ 0.40501965 -0.0033939   1.         ... -0.17088309 -0.15494682
  -0.09877457]
 ...
 [-0.07884583  0.02684759 -0.17088309 ...  1.          0.39632277
   0.00382471]
 [ 0.07740484  0.0412815  -0.15494682 ...  0.39632277  1.
   0.35172939]
 [-0.36456412 -0.1746839  -0.09877457 ...  0.00382471  0.35172939
   1.        ]]


In [8]:
def display_hypergraph(hypergraph):
    # Displaying the hypergraph
    n_nodes = len(hypergraph.nodes)
    n_edges = len(hypergraph.edges)
    print("Hypergraph number of nodes:", n_nodes)
    print("Hypergraph number of edges:", n_edges)
    print(hypergraph.shape)

    plt.subplots(figsize=(10,10))
    hnx.draw(hypergraph, with_edge_labels=False)
    plt.show()

## Visualization of some Hypergraphs built from various method implementations

In [35]:
def remove_duplicate_hyperedges(hypergraph_dict):
    unique_hyperedges = {}
    for hyperedge, vertices in hypergraph_dict.items():
        sorted_vertices = sorted(vertices)
        sorted_vertices_tuple = tuple(sorted_vertices)
        if sorted_vertices_tuple not in unique_hyperedges.values():
            unique_hyperedges[hyperedge] = sorted_vertices_tuple
    # print("Dictionary representation of the hypergraph:")
    # print(unique_hyperedges)
    return unique_hyperedges

### Fourier Clustering Method

In [10]:
# Using the Fourier Clustering Method

n_patient = 0
threshold = 0.4

def adjacency_matrix_to_hypergraph_dict(adjacency_matrix):
    hypergraph_dict = {}
    for i, row in enumerate(adjacency_matrix):
        hyperedge = []
        for j, val in enumerate(row):
            if val == 1:
                hyperedge.append(j)
        hypergraph_dict[i] = hyperedge
    return hypergraph_dict

def generate_adjacency_from_cluster_indices(cluster_indices):
    n_ROIs = len(cluster_indices)
    adjacency = np.zeros((n_ROIs, n_ROIs))
    for row_idx in range(n_ROIs):
        for col_idx in range(n_ROIs):
            if cluster_indices[row_idx] == cluster_indices[col_idx]:
                adjacency[row_idx, col_idx] = 1
    return adjacency

def generate_hypergraph_from_cluster(correlation_matrix, threshold, display=False):
    # Transforming correlation matrix into dissimilarity matrix
    dissimilarity_matrix = 1 - np.abs(correlation_matrix)
    dissimilarity_matrix[np.isnan(dissimilarity_matrix)] = 0
    # Performing Fourier clustering
    linkage_matrix = linkage(squareform(dissimilarity_matrix), method='complete')
    cluster_indices = fcluster(linkage_matrix, threshold, criterion='distance')
    # Generating hypergraph from the cluster indices
    adjacency = generate_adjacency_from_cluster_indices(cluster_indices)
    hg_dict = adjacency_matrix_to_hypergraph_dict(adjacency)
    # Reducing the amount of hyperedges by removing duplicated hyperedges
    hg_dict = remove_duplicate_hyperedges(hg_dict)
    # Creating HyperNetX hypergraph
    hg = hnx.Hypergraph(hg_dict)
    if display:
        display_hypergraph(hg)
    return hg, hg_dict

# Creating HyperNetX hypergraph from clustering method
path = f'ADNI_full/corr_matrices/corr_matrix_pearson/{corr_matrix_list[n_patient]}'
corr_matrix_patient = np.loadtxt(path, delimiter=',')
hg, hg_dict = generate_hypergraph_from_cluster(corr_matrix_patient, threshold, display=False)

/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:150: DeprecationWarning: misc_props_col and misc_cell_props_col will be deprecated; all public references to these arguments will be removed in a future release.
  warnings.warn(
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/hypergraph.py:563: FutureWarning: This method or function will be deprecated in a future release. Public references to this method or function will be removed from the Hypergraph API in a future release.
  self._nodes = self.E.restrict_to_levels([1])
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:618: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  elements = self._datafram

# Maximal Cliques Method

In [12]:
#Using the Maximal Cliques Method
def max_cliques(graph):
    cliques = list(nx.find_cliques(graph))
    max_clique_size = max(len(c) for c in cliques)
    summ = sum(1 for c in cliques)
    print(f"Number of cliques: {summ}")
    print(f"Maximum clique size: {max_clique_size}")
    return cliques, max_clique_size

def graph_to_hypergraph_max_cliques(graph, display=False):
    cliques, max_clique_size = max_cliques(graph)
    hg_dict = {i: set(clique) for i, clique in enumerate(cliques) if len(clique) == max_clique_size}
    hg = hnx.Hypergraph(hg_dict)
    if display:
        display_hypergraph(hg)
    return hg, hg_dict

# Creating an example HyperNetX hypergraph from max cliques method
threshold = 0.4
weight = False
age = False
sex = False
matrixprofile = True
method = 'pearson'

root = f'Raw_to_graph/ADNI_T_{threshold}_M_{method}_W{weight}_A{age}_S{sex}_MP{matrixprofile}'
dataset = f.Raw_to_Graph(root=root, threshold=threshold, method=method, weight=weight, age=age, sex=sex, matrixprofile=matrixprofile)
f.dataset_features_and_stats(dataset)
graph = f.r2g_to_nx(dataset[0])

hg, hg_dict = graph_to_hypergraph_max_cliques(graph)



Dataset: Raw_to_Graph(197):
Number of graphs: 197
Weighted: False
Threshold: 0.4
Correlation Method: pearson
Number of features: 461
Number of classes: 6

Data(edge_index=[2, 1410], weight=[1410], x=[116, 461], y=[1], num_nodes=116)
Number of nodes: 116
Number of edges: 1410
Average node degree: 12.16
Has isolated nodes: True
Has self-loops: True
Is undirected: True
Number of cliques: 242
Maximum clique size: 8


/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:150: DeprecationWarning: misc_props_col and misc_cell_props_col will be deprecated; all public references to these arguments will be removed in a future release.
  warnings.warn(
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/hypergraph.py:563: FutureWarning: This method or function will be deprecated in a future release. Public references to this method or function will be removed from the Hypergraph API in a future release.
  self._nodes = self.E.restrict_to_levels([1])
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:618: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  elements = self._datafram

# Coskewness Correlation Method

In [10]:
# Computing the coskewness cube

def coskew(df):
    num = 116
    coskew_cube = [[[0 for k in range(num)] for j in range(num)] for i in range(num)]
    for i in range(num):
        # v1 = [val for val in df[i]]
        v1 = df.iloc[:, i]
        meanv1 = np.mean(v1)
        std1 = np.std(v1)
        for j in range(num):
            # v2 = [val for val in df[j]]
            v2 = df.iloc[:, j]
            meanv2 = np.mean(v2)
            std2 = np.std(v2)
            for k in range(num):
                # v3 = [val for val in df[k]]
                v3 = df.iloc[:, k]
                meanv3 = np.mean(v3)
                std3 = np.std(v3)
                coskew_cube[i][j][k] = (np.mean((v1[i]-meanv1)*(v2[j]-meanv2)*(v3[k]-meanv3)))/(std1*std2*std3)

    print(np.min(coskew_cube))
    print(np.max(coskew_cube))
    ### Idk if it is necessary to normalize the results
    # Normalize the results
    max_abs_val = np.max(np.abs(coskew_cube))
    coskew_cube_normalized = coskew_cube / max_abs_val
    
    return coskew_cube_normalized

coskew_cube = coskew(time_series_df_list[0])
print(time_series_df_list[0])
print(time_series_df_list[0][0])
print(time_series_df_list[0][5])
print(time_series_df_list[0][115])

print(np.mean(coskew_cube))

print(len(coskew_cube))
print(len(coskew_cube[0]))
print(len(coskew_cube[0][0]))
print(coskew_cube[0][0][0])

-43.47079921407691
50.5763874573351
          0         1         2         3         4         5         6    \
0    0.264269  0.160402  0.573714 -0.016659  0.147470  0.327784  0.563114   
1    0.190860  0.211386 -0.334215  0.574008 -0.535597 -0.334685  0.021171   
2    0.728310  0.593930  0.083174  0.961234 -0.744761 -0.343953  0.225210   
3    1.173054  0.813959  0.850006  0.986218 -0.296675 -0.042714  0.487224   
4    0.520118  0.189850  0.294528  0.572696  0.235907 -0.238069 -0.158185   
..        ...       ...       ...       ...       ...       ...       ...   
134 -1.887287 -0.764243 -2.184397 -0.703209 -0.268133  0.508347 -1.634119   
135 -1.408172 -0.359410 -1.143153  0.383183  0.688927  0.152477 -0.765884   
136 -0.556275 -0.673630  0.305497  0.371132  1.935064  0.062150  0.366279   
137 -0.059465 -0.803845  0.755897  0.056427  1.975473  0.260254  0.696158   
138  0.099268  0.105493  0.253143  0.189465  0.149945  0.385766  0.264674   

          7         8         9    ... 

In [11]:
print(np.min(coskew_cube))
print(np.max(coskew_cube))

-0.8595077940421847
1.0


In [25]:
# Saving all the Coskewness Cubes
### Need to change this function to save the coskewness matrices in the correct format (i.e. here we have a 3D object??)
def save_coskew_matrices(coskew_cube, filename):
    with open(filename, 'wb') as fl:
        pkl.dump(coskew_cube, fl)
    return None

In [26]:
# Computing and saving the coskewness matrices
for i, time_series_df in enumerate(time_series_df_list):
    directory = f'ADNI_full/coskew_matrices/'
    patient_info = pd.read_csv('ADNI_full/patient_info.csv')
    os.makedirs(directory, exist_ok=True)
    idx_patient = patient_info['Subject ID'].values[i]
    filename = f'patient_{idx_patient}.pkl'
    if filename in os.listdir(directory):
        print(f"File {filename} already exists")
    else:
        filename = f'{directory}{filename}'
        coskew_cube = coskew(time_series_df)
        print('Done Processing patient nbr', i)
        save_coskew_matrices(coskew_cube, filename)

File patient_002_S_0295.pkl already exists
-15.135520749140344
14.175189713926274
Done Processing patient nbr 1
-16.31380136021064
15.668340935698994
Done Processing patient nbr 2
-14.105529511467033
14.501096727702551
Done Processing patient nbr 3
-14.298696819838979
14.301743016134012
Done Processing patient nbr 4
-14.318309085100049
17.042686150648205
Done Processing patient nbr 5
-19.127749098746595
15.302826856048085
Done Processing patient nbr 6
-16.53403094148009
24.011272129370795
Done Processing patient nbr 7
-37.69169222052089
34.253955163983235
Done Processing patient nbr 8
-58.375497166046216
49.364725760467635
Done Processing patient nbr 9
-14.91873222680838
16.120593228496947
Done Processing patient nbr 10
-27.45366232595899
20.627740267213238
Done Processing patient nbr 11
-11.093261670070056
10.624649315076526
Done Processing patient nbr 12
-12.74824850114439
10.428687567172574
Done Processing patient nbr 13
-11.905879123345619
10.607696998554035
Done Processing patient

/var/folders/wh/snjqmpvj7qv64q1ngy8zvqbh0000gn/T/ipykernel_13370/3764335747.py:21: RuntimeWarning: invalid value encountered in scalar divide
  coskew_cube[i][j][k] = (np.mean((v1[i]-meanv1)*(v2[j]-meanv2)*(v3[k]-meanv3)))/(std1*std2*std3)


nan
nan
Done Processing patient nbr 33
-21.800965427109794
31.49660004035883
Done Processing patient nbr 34
-32.38800904897185
40.157332840544825
Done Processing patient nbr 35
-17.517894594958953
16.815027499306744
Done Processing patient nbr 36
-19.89767536697721
17.636482806095
Done Processing patient nbr 37
-8.085578103423735
7.671445777442457
Done Processing patient nbr 38
-13.397826108509237
14.968627297219133
Done Processing patient nbr 39
-18.96267964314969
22.100367002221482
Done Processing patient nbr 40
-21.481441231912427
16.73598163372027
Done Processing patient nbr 41
-19.988107344361023
18.303316082243104
Done Processing patient nbr 42
-20.294502120673418
23.25494709767205
Done Processing patient nbr 43
-16.982576101511633
17.75373439120876
Done Processing patient nbr 44
-22.951635008887138
21.799562959845908
Done Processing patient nbr 45
-17.89828038157312
21.427942339443074
Done Processing patient nbr 46
-36.61801580064917
40.289732996503886
Done Processing patient nb

/var/folders/wh/snjqmpvj7qv64q1ngy8zvqbh0000gn/T/ipykernel_13370/3764335747.py:21: RuntimeWarning: invalid value encountered in scalar divide
  coskew_cube[i][j][k] = (np.mean((v1[i]-meanv1)*(v2[j]-meanv2)*(v3[k]-meanv3)))/(std1*std2*std3)


nan
nan
Done Processing patient nbr 106
-14.185270391157497
10.338115765856884
Done Processing patient nbr 107
-11.742508597701077
11.016448357374651
Done Processing patient nbr 108
-15.612910744027245
14.875623283014047
Done Processing patient nbr 109
-29.360261515578106
25.54942293166277
Done Processing patient nbr 110
-11.184452323952216
9.313193347809982
Done Processing patient nbr 111
-30.227033408759205
23.75337086389437
Done Processing patient nbr 112
-10.110784402903343
12.788021986469719
Done Processing patient nbr 113
-13.026858301360754
10.821163610741396
Done Processing patient nbr 114
-9.048892318627543
9.77282279304634
Done Processing patient nbr 115
-25.226204272595233
38.688583235849734
Done Processing patient nbr 116
-56.69174165085529
49.17161973400488
Done Processing patient nbr 117
-11.90591364296485
12.773436615207556
Done Processing patient nbr 118
-21.69069651271064
19.15720140179023
Done Processing patient nbr 119
-21.634415219172087
20.662147359675
Done Process

In [36]:
# Building a hypergraph from a coskewness cube
def adjacency_cube_to_hypergraph_dict(adjacency_cube):
    hg_dict = {}
    for i, row in enumerate(adjacency_cube):
        for j in range(len(row)):
            hyperedge = []
            for k, val in enumerate(row[j]):
                if val == 1:
                    if j not in hyperedge:
                        hyperedge.append(j)
                    hyperedge.append(k)
        hg_dict[i] = hyperedge
    return hg_dict

def coskewness_cube_to_hypergraph(coskewness_cube, threshold, display=False):
    adjacency_cube = np.zeros_like(coskewness_cube)
    for i in range(len(coskewness_cube)):
        for j in range(len(coskewness_cube[i])):
            for k in range(len(coskewness_cube[i][j])):
                if np.abs(coskewness_cube[i][j][k]) > threshold:
                    print(coskewness_cube[i][j][k])
                    adjacency_cube[i][j][k] = 1
    print(np.max(np.abs(adjacency_cube)))
    hg_dict = adjacency_cube_to_hypergraph_dict(adjacency_cube)
    # hg_dict = remove_duplicate_hyperedges(hg_dict)
    print(hg_dict)
    hg = hnx.Hypergraph(hg_dict)
    if display:
        display_hypergraph(hg)
    return hg, hg_dict

# Creating an example HyperNetX hypergraph from coskewness matrix
threshold = 0.4
filename = f'ADNI_full/coskew_matrices/patient_130_S_4883.pkl'
coskewness_cube = pkl.load(open(filename, 'rb'))
hg, hg_dict = coskewness_cube_to_hypergraph(coskewness_cube, threshold)
display_hypergraph(hg)

-0.5380079413243632
-0.4012674296681977
-0.5440296963682203
0.4851929138521989
-0.5440296963682202
0.4851929138521987
-0.7375836877048054
0.5306042646888326
-0.449403274227018
-0.43048091410157807
-0.4637794281747403
0.5010681851490908
0.5306042646888326
-0.44940327422701803
-0.43048091410157807
-0.4637794281747403
0.5010681851490909
-0.43969526919859514
0.4851929138521989
-0.4327193262131249
0.4851929138521988
-0.4327193262131249
0.6578140513953201
-0.47321944188967413
0.40080033419599
0.41362170786446356
-0.44687768776977027
-0.47321944188967424
0.40080033419599004
0.41362170786446356
-0.44687768776977027
-0.5001433227538149
-0.5380079413243632
-0.5440296963682202
0.4851929138521987
-0.7375836877048054
0.5306042646888326
-0.449403274227018
-0.43048091410157807
-0.4637794281747403
0.5010681851490908
-0.43969526919859514
0.4851929138521988
-0.4327193262131249
0.6578140513953201
-0.47321944188967413
0.40080033419599
0.41362170786446356
-0.44687768776977027
-0.5001433227538149
-0.5380079

/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:150: DeprecationWarning: misc_props_col and misc_cell_props_col will be deprecated; all public references to these arguments will be removed in a future release.
  warnings.warn(
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/hypergraph.py:563: FutureWarning: This method or function will be deprecated in a future release. Public references to this method or function will be removed from the Hypergraph API in a future release.
  self._nodes = self.E.restrict_to_levels([1])


KeyError: '[1] not in index'

# Saving the Hypergraphs for all three methods

In [21]:
# Defining a function to save hypergraphs
def save_hypergraph(hg_dict, directory, method, threshold, id):
    dir = f'{directory}/{method}/thresh_{threshold}'
    if not os.path.exists(dir):
        os.makedirs(dir)
    with open(f'{dir}/{id}_{method}_{threshold}.pkl', 'wb') as f:
        pkl.dump(hg_dict, f)
    return

In [28]:
time_series_folder = 'ADNI_full/time_series'
threshold = 0.4

for i, file_name in enumerate(os.listdir(time_series_folder)):
    patient_id = file_name[3:13]
    print(f'Processing patient {patient_id}')
    # for method in ['fourier_cluster', 'maximal_clique', 'coskewness']:
    # for method in ['maximal_clique', 'fourier_cluster']:
    for method in ['coskewness']:
        if method == 'fourier_cluster':
            path = f'ADNI_full/corr_matrices/corr_matrix_pearson/{corr_matrix_list[i]}'
            corr_matrix_patient = np.loadtxt(path, delimiter=',')
            _, hg_dict = generate_hypergraph_from_cluster(corr_matrix_patient, threshold=threshold)
        elif method == 'maximal_clique':
            method_corr = 'pearson'
            root = f'Raw_to_graph/ADNI_T_{threshold}_M_{method_corr}_WFalse_AFalse_SFalse_MPTrue'
            dataset = f.Raw_to_Graph(root=root, threshold=threshold, method=method_corr, weight=False, age=False, sex=False, matrixprofile=True)
            graph = f.r2g_to_nx(dataset[0])
            _, hg_dict = graph_to_hypergraph_max_cliques(graph)
        elif method == 'coskewness':
            filename = f'ADNI_full/coskew_matrices/patient_{patient_id}.pkl'
            coskew_cube = pkl.load(open(filename, 'rb'))
            _, hg_dict = coskewness_cube_to_hypergraph(coskew_cube, threshold=threshold)
        save_hypergraph(hg_dict, 'Hypergraphs', method, threshold, patient_id)
        print(f'Patient {patient_id} processed and saved for the {method} method with threshold {threshold}')

Processing patient 130_S_4883
{0: ()}


/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:150: DeprecationWarning: misc_props_col and misc_cell_props_col will be deprecated; all public references to these arguments will be removed in a future release.
  warnings.warn(
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/hypergraph.py:563: FutureWarning: This method or function will be deprecated in a future release. Public references to this method or function will be removed from the Hypergraph API in a future release.
  self._nodes = self.E.restrict_to_levels([1])


KeyError: '[1] not in index'

# Building the Raw_to_Hypergraph class

In [60]:
# Defining a class to preprocess raw data into a format suitable for training Graph Neural Networks (GNNs).
## With the possibility of assigning weight to edges, adding the age feature, sex feature, and matrixe profiling.

class Raw_to_Hypergraph(InMemoryDataset):
    def __init__(self, root, hg_data_path, method, weight, threshold, age=False, sex=False, transform=None, pre_transform=None):
        self.method = method
        self.weight = weight
        self.threshold = threshold
        self.age = age
        self.sex = sex
        self.hg_data_path = hg_data_path
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def processed_file_names(self):
        return ['data.pt']

    # This function is used to process the raw data into a format suitable for GNNs, by constructing graphs out of the connectivity matrices.
    def process(self):
        # Loading the prebuilt hypergraphs and the correlation matrices
        hg_dict_list = f.load_hg_dict(self.hg_data_path)
        full_corr_path_lists = f.corr_matrix_paths()
        corr_matrix_list = full_corr_path_lists['pearson']

        # Generating the diagnostic file from the diagnostic_label.csv file
        diagnostic_label = np.loadtxt('ADNI_full/diagnostic_label.csv', dtype=str, delimiter=',')
        # Combining the 'EMCI', 'LMCI' and 'MCI' diagnostics into a single 'MCI' label for simplicity, then one-hot encoding the diagnostics
        diagnostic_label = f.combine_diag_labels(diagnostic_label)

        graphs=[]
        for patient_idx, patient_hg in enumerate(hg_dict_list):
            # Create a NetworkX graph from the hypergraph matrix
            hypergraph = hnx.Hypergraph(patient_hg)

            # Adding the matrix profiling features to the feature array
            patient_matrix = corr_matrix_list[patient_idx]
            path = f'ADNI_full/matrix_profiles/matrix_profile_pearson/{patient_matrix}'
            if patient_matrix.endswith('.DS_Store'):
                continue  # Skip hidden system files like .DS_Store
            with open(path, "rb") as fl:
                patient_dict = pkl.load(fl)
            # combine dimensions
            features = np.array(patient_dict['mp']).reshape(len(patient_dict['mp']),-1)
            features = features.astype(np.float32)

            # Loading the atlas, atlas labels and nbr_ROIS
            _, _, _, nbr_ROIs = f.gen_atlas_labels()    
            if self.age:
                # Loading the age feature of patients to use as a node feature
                ages = np.loadtxt('ADNI_full/age.csv', delimiter=',')
                min_age = np.min(ages)
                max_age = np.max(ages)
                # Extracting the age feature of the patient
                patient_age = ages[patient_idx]
                age_norm = (patient_age - min_age) / (max_age - min_age)
                # Making the age array the same size as the other arrays
                age_array = np.full((nbr_ROIs,), age_norm)
                features = np.concatenate((features, age_array), axis=-1)
            if self.sex:
                # Prepocessing the sex feature of patients to use as a node feature. Here, 0 represents male patients and 1 represents female patients
                sex = np.loadtxt('ADNI_full/sex.csv', dtype=str, delimiter=',')
                for patient in range(len(sex)):
                    if sex[patient] == 'M':
                        sex[patient] = 0
                    else:
                        sex[patient] = 1
                # Extracting the sex feature of the patient
                patient_sex = int(sex[patient_idx])
                # Making the sex array the same size as the other arrays
                sex_array = np.full((nbr_ROIs,), patient_sex)
                features = np.concatenate((features, sex_array), axis=-1)

            # Concatenate the degree, participation coefficient, betweenness centrality, local efficiency, and ratio of local to global efficiency arrays to form a single feature vector
            x = torch.tensor(features, dtype=torch.float)

            # Create a Pytorch Geometric Data object
            edge_index0 = []
            edge_index1 = []
            i = 0
            for hyperedge, nodes in hypergraph.incidence_dict.items():
                edge_index0 = np.concatenate((edge_index0, nodes), axis=0)
                for j in range(len(nodes)):
                    edge_index1.append(i)
                i += 1
            edge_index = np.stack([edge_index0.astype(int), edge_index1], axis=0)
            y = torch.tensor(float(diagnostic_label[patient_idx]))
            hg_data = Data(x=x, edge_index=torch.tensor(edge_index, dtype=torch.long), y=y)
            graphs.append(hg_data)

        data, slices = self.collate(graphs)
        torch.save((data, slices), self.processed_paths[0])

In [61]:
# Testing the class Raw_to_Hypergraph with one example and saving it
threshold = 0.4
age = False
sex = False
method = 'maximal_clique'
if method == 'coskewness':
    weight = True
else:
    weight = False

hg_data_path = f'Hypergraphs/{method}/thresh_{threshold}'

root = f'Raw_to_hypergraph/ADNI_T_{threshold}_M_{method}_W{weight}_A{age}_S{sex}_MPTrue'
dataset = Raw_to_Hypergraph(root=root, hg_data_path=hg_data_path, method=method, weight=weight, threshold=threshold, age=age, sex=sex)
print(dataset[0].edge_index)

Processing...
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:150: DeprecationWarning: misc_props_col and misc_cell_props_col will be deprecated; all public references to these arguments will be removed in a future release.
  warnings.warn(
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/hypergraph.py:563: FutureWarning: This method or function will be deprecated in a future release. Public references to this method or function will be removed from the Hypergraph API in a future release.
  self._nodes = self.E.restrict_to_levels([1])
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:618: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  elements = 

{4: {0, 1, 42, 43, 81, 54, 56, 57}, 164: {96, 42, 43, 44, 45, 46, 48, 49}, 165: {42, 43, 44, 45, 46, 47, 48, 49}, 166: {42, 43, 44, 46, 47, 48, 49, 55}}
{4: [0, 1, 42, 43, 81, 54, 56, 57], 164: [96, 42, 43, 44, 45, 46, 48, 49], 165: [42, 43, 44, 45, 46, 47, 48, 49], 166: [42, 43, 44, 46, 47, 48, 49, 55]}
[ 0.  1. 42. 43. 81. 54. 56. 57. 96. 42. 43. 44. 45. 46. 48. 49. 42. 43.
 44. 45. 46. 47. 48. 49. 42. 43. 44. 46. 47. 48. 49. 55.]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3]
[[ 0  1 42 43 81 54 56 57 96 42 43 44 45 46 48 49 42 43 44 45 46 47 48 49
  42 43 44 46 47 48 49 55]
 [ 0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2
   3  3  3  3  3  3  3  3]]
{4: {0, 1, 42, 43, 81, 54, 56, 57}, 164: {96, 42, 43, 44, 45, 46, 48, 49}, 165: {42, 43, 44, 45, 46, 47, 48, 49}, 166: {42, 43, 44, 46, 47, 48, 49, 55}}
{4: [0, 1, 42, 43, 81, 54, 56, 57], 164: [96, 42, 43, 44, 45, 46, 48, 49], 165: [42, 43, 44, 45, 46, 47, 48, 4

/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:618: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  elements = self._dataframe.groupby(col1)[col2].unique().to_dict()
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:150: DeprecationWarning: misc_props_col and misc_cell_props_col will be deprecated; all public references to these arguments will be removed in a future release.
  warnings.warn(
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/hypergraph.py:563: FutureWarning: This method or function will be deprecated in a future release. Public references to this method or function will be removed from the Hypergraph API in a future release.
  self

[ 0.  1. 42. 43. 81. 54. 56. 57. 96. 42. 43. 44. 45. 46. 48. 49. 42. 43.
 44. 45. 46. 47. 48. 49. 42. 43. 44. 46. 47. 48. 49. 55.]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3]
[[ 0  1 42 43 81 54 56 57 96 42 43 44 45 46 48 49 42 43 44 45 46 47 48 49
  42 43 44 46 47 48 49 55]
 [ 0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2
   3  3  3  3  3  3  3  3]]
{4: {0, 1, 42, 43, 81, 54, 56, 57}, 164: {96, 42, 43, 44, 45, 46, 48, 49}, 165: {42, 43, 44, 45, 46, 47, 48, 49}, 166: {42, 43, 44, 46, 47, 48, 49, 55}}
{4: [0, 1, 42, 43, 81, 54, 56, 57], 164: [96, 42, 43, 44, 45, 46, 48, 49], 165: [42, 43, 44, 45, 46, 47, 48, 49], 166: [42, 43, 44, 46, 47, 48, 49, 55]}
[ 0.  1. 42. 43. 81. 54. 56. 57. 96. 42. 43. 44. 45. 46. 48. 49. 42. 43.
 44. 45. 46. 47. 48. 49. 42. 43. 44. 46. 47. 48. 49. 55.]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3]
[[ 0  1 42 43 81 54 56 57 96 42 43 

/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:150: DeprecationWarning: misc_props_col and misc_cell_props_col will be deprecated; all public references to these arguments will be removed in a future release.
  warnings.warn(
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/hypergraph.py:563: FutureWarning: This method or function will be deprecated in a future release. Public references to this method or function will be removed from the Hypergraph API in a future release.
  self._nodes = self.E.restrict_to_levels([1])
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:618: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  elements = self._datafram

...

/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:150: DeprecationWarning: misc_props_col and misc_cell_props_col will be deprecated; all public references to these arguments will be removed in a future release.
  warnings.warn(
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/hypergraph.py:563: FutureWarning: This method or function will be deprecated in a future release. Public references to this method or function will be removed from the Hypergraph API in a future release.
  self._nodes = self.E.restrict_to_levels([1])
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:618: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  elements = self._datafram

{4: {0, 1, 42, 43, 81, 54, 56, 57}, 164: {96, 42, 43, 44, 45, 46, 48, 49}, 165: {42, 43, 44, 45, 46, 47, 48, 49}, 166: {42, 43, 44, 46, 47, 48, 49, 55}}
{4: [0, 1, 42, 43, 81, 54, 56, 57], 164: [96, 42, 43, 44, 45, 46, 48, 49], 165: [42, 43, 44, 45, 46, 47, 48, 49], 166: [42, 43, 44, 46, 47, 48, 49, 55]}
[ 0.  1. 42. 43. 81. 54. 56. 57. 96. 42. 43. 44. 45. 46. 48. 49. 42. 43.
 44. 45. 46. 47. 48. 49. 42. 43. 44. 46. 47. 48. 49. 55.]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3]
[[ 0  1 42 43 81 54 56 57 96 42 43 44 45 46 48 49 42 43 44 45 46 47 48 49
  42 43 44 46 47 48 49 55]
 [ 0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2
   3  3  3  3  3  3  3  3]]
{4: {0, 1, 42, 43, 81, 54, 56, 57}, 164: {96, 42, 43, 44, 45, 46, 48, 49}, 165: {42, 43, 44, 45, 46, 47, 48, 49}, 166: {42, 43, 44, 46, 47, 48, 49, 55}}
{4: [0, 1, 42, 43, 81, 54, 56, 57], 164: [96, 42, 43, 44, 45, 46, 48, 49], 165: [42, 43, 44, 45, 46, 47, 48, 4

/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:150: DeprecationWarning: misc_props_col and misc_cell_props_col will be deprecated; all public references to these arguments will be removed in a future release.
  warnings.warn(
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/hypergraph.py:563: FutureWarning: This method or function will be deprecated in a future release. Public references to this method or function will be removed from the Hypergraph API in a future release.
  self._nodes = self.E.restrict_to_levels([1])
/Users/mathilde/anaconda3/envs/alzheimers-cl/lib/python3.11/site-packages/hypernetx/classes/entityset.py:618: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  elements = self._datafram

{4: {0, 1, 42, 43, 81, 54, 56, 57}, 164: {96, 42, 43, 44, 45, 46, 48, 49}, 165: {42, 43, 44, 45, 46, 47, 48, 49}, 166: {42, 43, 44, 46, 47, 48, 49, 55}}
{4: [0, 1, 42, 43, 81, 54, 56, 57], 164: [96, 42, 43, 44, 45, 46, 48, 49], 165: [42, 43, 44, 45, 46, 47, 48, 49], 166: [42, 43, 44, 46, 47, 48, 49, 55]}
[ 0.  1. 42. 43. 81. 54. 56. 57. 96. 42. 43. 44. 45. 46. 48. 49. 42. 43.
 44. 45. 46. 47. 48. 49. 42. 43. 44. 46. 47. 48. 49. 55.]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3]
[[ 0  1 42 43 81 54 56 57 96 42 43 44 45 46 48 49 42 43 44 45 46 47 48 49
  42 43 44 46 47 48 49 55]
 [ 0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2
   3  3  3  3  3  3  3  3]]
{4: {0, 1, 42, 43, 81, 54, 56, 57}, 164: {96, 42, 43, 44, 45, 46, 48, 49}, 165: {42, 43, 44, 45, 46, 47, 48, 49}, 166: {42, 43, 44, 46, 47, 48, 49, 55}}
{4: [0, 1, 42, 43, 81, 54, 56, 57], 164: [96, 42, 43, 44, 45, 46, 48, 49], 165: [42, 43, 44, 45, 46, 47, 48, 4

Done!


In [ ]:
# Testing the class Raw_to_Graph with different thresholds, weights and methods and saving it
thresholds = [0.4, 0.5, 0.6]
age = False
sex = False
methods = ['fourier_cluster', 'maximal_clique', 'coskewness']
# methods = ['maximal_clique']
if method == 'coskewness':
    weight = True
else:
    weight = False

for method in methods:
    for threshold in thresholds:
        hg_data_path = f'Hypergraphs/{method}/thresh_{threshold}'
        root = f'Raw_to_hypergraph/ADNI_T_{threshold}_M_{method}_W{weight}_A{age}_S{sex}_MPTrue'
        dataset = Raw_to_Hypergraph(root=root, hg_data_path=hg_data_path, method=method, weight=weight, threshold=threshold, age=age, sex=sex)